In [ ]:
import re
import nltk
import time
from nltk.corpus import stopwords
import nltk.data
from gensim.models import word2vec
from gensim.models import Word2Vec

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv( "../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", \
                    header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "../input/word2vec-nlp-tutorial/testData.tsv.zip", \
                   header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "../input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip", \
                              header=0, delimiter="\t", quoting=3 )

In [ ]:
def review_to_wordlist (review, remove_stopwords=False):
    # 1. Remove HTML
    review_text=BeautifulSoup(review).get_text()
    # 2. Remove non-letters
    review_text=re.sub( "[^a-zA-Z]"," ", review_text)
    # 3. convert to small letters and split to words    
    words=review_text.lower().split()
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stop=set(stopwords.words('english'))
        words=[w for w in words if not w in stop]
    return (words)

In [ ]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [ ]:
sentences = []  # Initialize an empty list of sentences


for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)


for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

A minor detail to note is the difference between the "+=" and "append" when it comes to Python lists. In many applications the two are interchangeable, but here they are not. If you are appending a list of lists to another list of lists, "append" will only append the first list; you need to use "+=" in order to join all of the lists at once.

In [ ]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# # Initialize and train the model (this will take some time)
# from gensim.models import word2vec
# print ("Training model...")
# model = word2vec.Word2Vec(sentences, workers=num_workers, \
#             size=num_features, min_count = min_word_count, \
#             window = context, sample = downsampling)

# # If you don't plan to train the model any further, calling 
# # init_sims will make the model much more memory-efficient.
# model.init_sims(replace=True)

# # It can be helpful to create a meaningful model name and 
# # save the model for later use. You can load it later using Word2Vec.load()
# model_name = "300features_40minwords_10context"
# model.save(model_name)

## Training and Saving Your Model
With the list of nicely parsed sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec API documentation as well as the Google documentation. 

**Architecture**: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.

**Training algorithm**: Hierarchical softmax (default) or negative sampling. For us, the default worked well.

**Downsampling of frequent words**: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.

**Word vector dimensionality**: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.

**Context / window size**: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).

**Worker threads**: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.

**Minimum word count**: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

Choosing parameters is not easy, but once we have chosen our parameters, creating a Word2Vec model is straightforward:

In [ ]:
model=word2vec.Word2Vec(sentences, workers=4, size=300, min_count=40, window=10, sample=1e-3)
model.init_sims(replace=True)
model_name="300features_40minwords_10context"
model.save(model_name)

In [ ]:
model = Word2Vec.load("300features_40minwords_10context")

In [ ]:
len(model.wv.index2word)

In [ ]:
def makeFeatureVec(words, model, num_features):
      
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
     
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
     
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [ ]:
def getAvgFeatureVecs(reviews, model, num_features):
   
     
    # Initialize a counter
    counter = 0
    
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    
    # Loop through the reviews
    for review in reviews:
       # Print a status message every 1000th review
       if counter%1000 == 0:
            print(f"Review {counter} of {len(reviews)}")
            
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        
       # Increment the counter
       counter = counter + 1
        
    return reviewFeatureVecs

In [ ]:
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))

trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist(review,remove_stopwords=True))
print("Finished creating average feature vecs for test reviews")

testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

In [ ]:
def RandomForest(X_train, y_train, X_test):
    
    model=RandomForestClassifier(n_estimators=150)
    model.fit(X_train, y_train)
    predictions=model.predict(X_test)
    
    return predictions

In [ ]:
sub=pd.DataFrame(data={'id':test['id'],\
                       'sentiment':predictions})
sub.to_csv('submission2.csv', index=False, quoting=3)

In [ ]:
def LogisticReg(X_train, y_train, X_test):
    model=LogisticRegression(C=1)
    model.fit(X_train, y_train)
    result=model.predict(X_test)
    return result

In [ ]:
sub=pd.DataFrame(data={'id':test['id'],
                         'sentiment':result})
sub.to_csv('submission_logreg.csv',index=False,quoting=3)

In [ ]:
model.wv.syn0

In [ ]:
start=time.time()

word_vectors = model.wv.syn0
num_clusters = int(word_vectors.shape[0] / 5)

kmeans_clustering=KMeans(n_clusters=num_clusters)
idx=kmeans_clustering.fit_predict(word_vectors)

print(f'Time', time.time()-start)

In [ ]:
word_centroid_map=dict(zip(model.wv.index2word, idx))

In [ ]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip(model.wv.index2word, idx))
word_centroid_map

In [ ]:
# For the first 10 clusters
for cluster in range(0,10):
    
    # Print the cluster number  
    print(f"\nCluster {cluster}")
    
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if(list(word_centroid_map.values())[i] == cluster):
            words.append(list(word_centroid_map.keys())[i])
    print(words)

In [ ]:
max( word_centroid_map.values() )

In [ ]:
def create_bag_of_centroids (wordlist, word_centroid_map):
    num_centroid=max(word_centroid_map.values()) + 1
    
    bag_of_centroids=np.zeros(num_centroid, dtype='float32')
    
    for word in wordlist:
        if word in word_centroid_map:
            index=word_centroid_map[word]
            bag_of_centroids[index]+=1
    return bag_of_centroids

In [ ]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [ ]:
pred=RandomForest(train_centroids, train['sentiment'], test_centroids)

In [ ]:
sub=pd.DataFrame(data={'id':test['id'],
                      'sentiment':pred})
sub.to_csv('bag_of_words.csv', index=False, quoting=3)

In [ ]:
model2=LogisticRegression(C=1)
model2.fit(train_centroids, train['sentiment'])
result=model2.predict(test_centroids)

In [ ]:
sub=pd.DataFrame(data={'id':test['id'],
                      'sentiment':result})
sub.to_csv('bag_of_words_log.csv', index=False, quoting=3)